
이 노트북은 다음에 대한 실험 결과가 실려 있습니다.

* 손실함수
* 학습률 감쇠
* 가중치 초기화
* 최적화 기법
* 드롭아웃

# `lincoln` 라이프러리 임포트하기

In [21]:
import sys
# 예제 파일 경로로 수정한 다음 주석 해제
# sys.path.append(r'/to/your/example_code/path/lincoln')

In [22]:
import numpy as np

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import lincoln
from lincoln.layers import Dense
from lincoln.losses import SoftmaxCrossEntropy, MeanSquaredError
from lincoln.optimizers import Optimizer, SGD, SGDMomentum
from lincoln.activations import Sigmoid, Tanh, Linear, ReLU
from lincoln.network import NeuralNetwork
from lincoln.train import Trainer
from lincoln.utils import mnist
from lincoln.utils.np_utils import softmax

In [16]:
#mnist.init() # 최초 실행시 주석 해제, 이후 다시 주석 처리할 것

In [17]:
X_train, y_train, X_test, y_test = mnist.load()

In [18]:
num_labels = len(y_train)
num_labels

60000

In [19]:
# 원-핫 인코딩
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

# MNIST 데모

# 데이터를 평균 0 분산 1로 정규화

In [20]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)

In [21]:
np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test)

(-33.318421449829934,
 221.68157855017006,
 -33.318421449829934,
 221.68157855017006)

In [22]:
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [23]:
np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test)

(-0.424073894391566, 2.821543345689335, -0.424073894391566, 2.821543345689335)

In [25]:
def calc_accuracy_model(model, test_set):
    return print(f'''모델 검증을 위한 정확도: {np.equal(np.argmax(model.forward(test_set, inference=True), axis=1), y_test).sum() * 100.0 / test_set.shape[0]:.2f}%''')

## 소프트맥스 교차 엔트로피

### sigmoid 활성화 함수를 사용한 경우

In [29]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=False), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);
print()
calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.611
20에폭에서 검증 데이터에 대한 손실값: 0.424
30에폭에서 검증 데이터에 대한 손실값: 0.387
40에폭에서 검증 데이터에 대한 손실값: 0.373
50에폭에서 검증 데이터에 대한 손실값: 0.365

모델 검증을 위한 정확도: 72.60%


메모: 분류 모델의 출력을 평균제곱오차 손실로 정규화 하더라도 성능이 향상되지 않는다.

In [31]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(normalize=True), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.952

20에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 10 에폭까지 학습된 모델에서 계산된 0.952이다.
모델 검증을 위한 정확도: 41.73%


소프트맥스 교차 엔트로피 손실함수를 사용해야 하는 이유가 여기에 있다.

#### sigmoid 활성화 함수를 사용한 경우

In [32]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 130,
            eval_every = 1,
            seed=20190119,
            batch_size=60);
print()
calc_accuracy_model(model, X_test)

1에폭에서 검증 데이터에 대한 손실값: 1.285
2에폭에서 검증 데이터에 대한 손실값: 0.970
3에폭에서 검증 데이터에 대한 손실값: 0.836
4에폭에서 검증 데이터에 대한 손실값: 0.763
5에폭에서 검증 데이터에 대한 손실값: 0.712
6에폭에서 검증 데이터에 대한 손실값: 0.679
7에폭에서 검증 데이터에 대한 손실값: 0.651
8에폭에서 검증 데이터에 대한 손실값: 0.631
9에폭에서 검증 데이터에 대한 손실값: 0.617
10에폭에서 검증 데이터에 대한 손실값: 0.599
11에폭에서 검증 데이터에 대한 손실값: 0.588
12에폭에서 검증 데이터에 대한 손실값: 0.576
13에폭에서 검증 데이터에 대한 손실값: 0.568
14에폭에서 검증 데이터에 대한 손실값: 0.557
15에폭에서 검증 데이터에 대한 손실값: 0.550
16에폭에서 검증 데이터에 대한 손실값: 0.544
17에폭에서 검증 데이터에 대한 손실값: 0.537
18에폭에서 검증 데이터에 대한 손실값: 0.533
19에폭에서 검증 데이터에 대한 손실값: 0.529
20에폭에서 검증 데이터에 대한 손실값: 0.523
21에폭에서 검증 데이터에 대한 손실값: 0.517
22에폭에서 검증 데이터에 대한 손실값: 0.512
23에폭에서 검증 데이터에 대한 손실값: 0.507

24에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 23 에폭까지 학습된 모델에서 계산된 0.507이다.

모델 검증을 위한 정확도: 91.04%


####  ReLU 활성화 함수를 사용한 경우

In [33]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=ReLU()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);
print()
calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 6.434
20에폭에서 검증 데이터에 대한 손실값: 5.798

30에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 20 에폭까지 학습된 모델에서 계산된 5.798이다.

모델 검증을 위한 정확도: 75.99%


In [34]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGD(0.1))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);
print()
calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.628
20에폭에서 검증 데이터에 대한 손실값: 0.567
30에폭에서 검증 데이터에 대한 손실값: 0.545
40에폭에서 검증 데이터에 대한 손실값: 0.544
50에폭에서 검증 데이터에 대한 손실값: 0.541

모델 검증을 위한 정확도: 91.05%


## 모멘텀을 적용한 확률적 경사 하강법

In [35]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optim = SGDMomentum(0.1, momentum=0.9)

trainer = Trainer(model, SGDMomentum(0.1, momentum=0.9))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 1,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

1에폭에서 검증 데이터에 대한 손실값: 0.615
2에폭에서 검증 데이터에 대한 손실값: 0.489
3에폭에서 검증 데이터에 대한 손실값: 0.447

4에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 3 에폭까지 학습된 모델에서 계산된 0.447이다.
모델 검증을 위한 정확도: 92.07%


In [36]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optim = SGD(0.1)

optim = SGDMomentum(0.1, momentum=0.9)

trainer = Trainer(model, SGDMomentum(0.1, momentum=0.9))
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.390
20에폭에서 검증 데이터에 대한 손실값: 0.363
30에폭에서 검증 데이터에 대한 손실값: 0.322

40에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 30 에폭까지 학습된 모델에서 계산된 0.322이다.
모델 검증을 위한 정확도: 95.35%


## 여러 가지 학습률 감쇠 기법

In [37]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.15, momentum=0.9, final_lr = 0.05, decay_type='linear')

trainer = Trainer(model, optimizer)
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.397
20에폭에서 검증 데이터에 대한 손실값: 0.300
30에폭에서 검증 데이터에 대한 손실값: 0.297

40에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 30 에폭까지 학습된 모델에서 계산된 0.297이다.
모델 검증을 위한 정확도: 95.83%


In [38]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh()),
            Dense(neurons=10, 
                  activation=Linear())],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.2, 
                        momentum=0.9, 
                        final_lr = 0.05, 
                        decay_type='exponential')

trainer = Trainer(model, optimizer)
trainer.fit(X_train, train_labels, X_test, test_labels,
            epochs = 50,
            eval_every = 10,
            seed=20190119,
            batch_size=60);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.439
20에폭에서 검증 데이터에 대한 손실값: 0.324
30에폭에서 검증 데이터에 대한 손실값: 0.288
40에폭에서 검증 데이터에 대한 손실값: 0.283

50에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 40 에폭까지 학습된 모델에서 계산된 0.283이다.
모델 검증을 위한 정확도: 96.03%


## 가중치 초기화 방법에 대한 실험

In [39]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

optimizer = SGDMomentum(0.15, momentum=0.9, final_lr = 0.05, decay_type='linear')

trainer = Trainer(model, optimizer)
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.329
20에폭에서 검증 데이터에 대한 손실값: 0.230

30에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 20 에폭까지 학습된 모델에서 계산된 0.230이다.
모델 검증을 위한 정확도: 96.73%


In [40]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.413
20에폭에서 검증 데이터에 대한 손실값: 0.287
30에폭에서 검증 데이터에 대한 손실값: 0.242

40에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 30 에폭까지 학습된 모델에서 계산된 0.242이다.
모델 검증을 위한 정확도: 96.60%


## 드롭아웃

In [41]:
model = NeuralNetwork(
    layers=[Dense(neurons=89, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 50,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.272
20에폭에서 검증 데이터에 대한 손실값: 0.221
30에폭에서 검증 데이터에 대한 손실값: 0.206
40에폭에서 검증 데이터에 대한 손실값: 0.193

50에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 40 에폭까지 학습된 모델에서 계산된 0.193이다.
모델 검증을 위한 정확도: 96.84%


## 드롭아웃 적용 여부에 따른 딥러닝 성능 차이

In [42]:
model = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot",
                  dropout=0.8),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 100,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.320
20에폭에서 검증 데이터에 대한 손실값: 0.259
30에폭에서 검증 데이터에 대한 손실값: 0.257
40에폭에서 검증 데이터에 대한 손실값: 0.229
50에폭에서 검증 데이터에 대한 손실값: 0.209

60에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 50 에폭까지 학습된 모델에서 계산된 0.209이다.
모델 검증을 위한 정확도: 96.89%


In [43]:
model = NeuralNetwork(
    layers=[Dense(neurons=178, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=46, 
                  activation=Tanh(),
                  weight_init="glorot"),
            Dense(neurons=10, 
                  activation=Linear(),
                  weight_init="glorot")],
            loss = SoftmaxCrossEntropy(), 
seed=20190119)

trainer = Trainer(model, SGDMomentum(0.2, momentum=0.9, final_lr = 0.05, decay_type='exponential'))
trainer.fit(X_train, train_labels, X_test, test_labels,
       epochs = 100,
       eval_every = 10,
       seed=20190119,
           batch_size=60,
           early_stopping=True);

calc_accuracy_model(model, X_test)

10에폭에서 검증 데이터에 대한 손실값: 0.496
20에폭에서 검증 데이터에 대한 손실값: 0.365
30에폭에서 검증 데이터에 대한 손실값: 0.341
40에폭에서 검증 데이터에 대한 손실값: 0.304
50에폭에서 검증 데이터에 대한 손실값: 0.269
60에폭에서 검증 데이터에 대한 손실값: 0.261
70에폭에서 검증 데이터에 대한 손실값: 0.249

80에폭에서 손실값이 증가했다. 마지막으로 측정한 손실값은 70 에폭까지 학습된 모델에서 계산된 0.249이다.
모델 검증을 위한 정확도: 96.25%
